In [4]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from collections import Counter

# Load model
model = load_model('facial_expressions_model.h5')

# Define class names
class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

# Start webcam
cap = cv2.VideoCapture(0)  # 0 = default camera

# Track emotions over time
emotion_history = []

# Haar Cascade (for face detection)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def analyze_emotions(history):
    """Analyze emotion history and return patient state message."""
    if not history:
        return "No data yet."

    counts = Counter(history)
    total = sum(counts.values())

    # Calculate relative frequencies
    sad_freq = counts.get("Sad", 0) / total
    fear_freq = counts.get("Fear", 0) / total
    happy_freq = counts.get("Happy", 0) / total
    angry_freq = counts.get("Angry", 0) / total
    neutral_freq = counts.get("Neutral", 0) / total

    # Rule-based analysis
    if sad_freq > 0.3 or fear_freq > 0.3:
        return "Signs of depression or anxiety detected."
    elif happy_freq < 0.1 and total > 20:  # enough samples collected
        return "Possible emotional distress (low happiness)."
    elif angry_freq > 0.25 or neutral_freq > 0.4:
        return "High stress or irritability suspected."
    else:
        return "Stable emotional state."

while True:
    ret, frame = cap.read()
    if not ret:
        print("Could not access the camera.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    last_message = "Waiting for detection..."

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_resized = cv2.resize(roi_gray, (48, 48))
        roi_normalized = roi_resized / 255.0
        roi_reshaped = np.reshape(roi_normalized, (1, 48, 48, 1))

        # Predict
        prediction = model.predict(roi_reshaped, verbose=0)
        predicted_index = np.argmax(prediction)
        predicted_class = class_names[predicted_index]
        confidence = int(prediction[0][predicted_index] * 100)

        # Save to history
        emotion_history.append(predicted_class)
        if len(emotion_history) > 200:  # keep recent history manageable
            emotion_history.pop(0)

        # Update message
        last_message = f"Detected: {predicted_class} ({confidence}%)"

        # Draw bounding box
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(frame, f"{predicted_class} ({confidence}%)", (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

    # Analyze patient state
    patient_state = analyze_emotions(emotion_history)

    # Draw bottom panel
    h, w, _ = frame.shape
    cv2.rectangle(frame, (0, h-70), (w, h), (0, 0, 0), -1)  # black bar

    cv2.putText(frame, last_message, (10, h-45),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.putText(frame, "Patient State: " + patient_state, (10, h-15),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # Show video
    cv2.imshow('Emotion Detection', frame)

    # Exit with 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Exiting...")
        break

cap.release()
cv2.destroyAllWindows()

Exiting...
